In [ ]:
from generators import LXM, GCL
import numpy as np
import simpy

lambdaArribos = 9
lambdaServicio = 10
tiempoTotal = 100
generador = LXM()


def exponencial(lambdaArgumento, generador):
    uniforme = generador.generar()
    return - np.log(1 - uniforme) / lambdaArgumento


class Satelite:
    def __init__(self, env, servicioDescarga, lambdaArgumento, generador):
        self.env = env
        self.lambdaArgumento = lambdaArgumento
        self.generador = generador
        self.servicioDescarga = servicioDescarga

    def atencion(self, id):
        tiempoAtencion = exponencial(self.lambdaArgumento, self.generador)
        servicio = self.servicioDescarga.request()
        yield servicio
        print("Satelite en uso por la solicitud con id " + str(id))
        N = 1
        largoCola = len(self.servicioDescarga.queue)
        if largoCola != 0:
            N = largoCola
        tiempoServicio = tiempoAtencion / N
        print("El tiempo de atencion obtenido es: " + str(tiempoAtencion)  + ", tiempoServicio: " + str(tiempoServicio))
        print("En la cola hay " + str(len(self.servicioDescarga.queue)))
        yield env.timeout(tiempoServicio)
        self.servicioDescarga.release(servicio)

class Solicitud:
    def __init__(self, env, satelite, id):
        self.env = env
        self.satelite = satelite
        self.id = id

    def tratarse(self):
        return self.satelite.atencion(self.id)



def generarArribos(lambdaArgumento, generador, intervaloDeTiempo, env, satelite):
    tiempoAcumulado = 0
    tiempoEntreArribos = []
    id = 0
    while (tiempoAcumulado < intervaloDeTiempo):
        tiempo = exponencial(lambdaArgumento, generador)
        if (tiempo + tiempoAcumulado > intervaloDeTiempo):
            break 
        nuevaSolicitud = Solicitud(env, satelite, id)
        id = 1 + id
        env.process(nuevaSolicitud.tratarse())
        yield env.timeout(tiempo) # pasa el tiempo en el ambiente

        tiempoAcumulado += tiempo
        tiempoEntreArribos.append(tiempo)



env = simpy.Environment()
servicioDescarga = simpy.Resource(env, capacity=1)
satelite = Satelite(env,servicioDescarga,lambdaServicio,generador)

env.process(generarArribos(lambdaArribos, generador, tiempoTotal, env, satelite))

env.run()